In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/home/jk394348/hyper-dl')
print(os.getcwd())

import pickle
from omegaconf import DictConfig
from config import run_training
from data import build_loaders
from plot import setup_dirs, dump_results


def build_dataset_path(seed, n_samples, dim, eps, curv, transform_dim):
    template = 'dim={},n_samples={},eps={},transform_dim={},curv={},seed={}.pkl'
    return template.format(int(dim), int(n_samples), eps, transform_dim, curv, seed)

/home/jk394348/hyper-dl


In [2]:
import pickle
from omegaconf import DictConfig
from config import run_training
from data import build_loaders
from plot import setup_dirs, dump_results


def build_dataset_path(seed, n_samples, dim, eps, curv, transform_dim):
    template = 'dim={},n_samples={},eps={},transform_dim={},curv={},seed={}.pkl'
    return template.format(int(dim), int(n_samples), eps, transform_dim, curv, seed)

In [3]:
import hydra
from hydra.utils import get_original_cwd
from hydra import initialize, compose

overrides = [
    'num_workers=2',
    'dataset_params.dim=1',
    #'~neptune_cfg',
    'epochs=400',
    'model=relu_d5_w320',
    #'scheduler.name=None',
    # 'model/head=hyperbolic_dist',
    # 'model.encoder.hidden_dims=[32,32]',
    # 'model.encoder.bias=True',
    'bs=4096',
    'optimizer.lr=0.016',
    'r_optimizer.lr=0.008'
]

with initialize(config_path="../conf"):
    cfg = compose(config_name="config.yaml", overrides=overrides)

In [4]:
cwd = os.getcwd()
print("Working directory : {}".format(cwd))

dataset_path = build_dataset_path(**cfg['dataset_params'])
dataset_path = os.path.join(cwd, 'datasets', dataset_path)

with open(dataset_path, 'rb') as f:
    datasets = pickle.load(f)
loaders = build_loaders(datasets, cfg['bs'], cfg['num_workers'])

results, run = run_training(loaders=loaders,
                            cfg=cfg,
                            **cfg)

run.stop()

Working directory : /home/jk394348/hyper-dl
https://app.neptune.ai/jkociniak/hyperbolic-dist-pred/e/HYPERDIST-2760
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Found device: cuda
[1] train loss (mean of batch losses): 3.1903996372767858
[1] val loss: 2.0795427734375
[2] train loss (mean of batch losses): 1.224188387625558
[2] val loss: 0.557686376953125
[3] train loss (mean of batch losses): 0.454265274483817
[3] val loss: 0.35017421264648435
[4] train loss (mean of batch losses): 0.3067190115792411
[4] val loss: 0.24490597534179687
[5] train loss (mean of batch losses): 0.1974021741594587
[5] val loss: 0.15146104431152344
[6] train loss (mean of batch losses): 0.12732552577427456
[6] val loss: 0.10561213989257813
[7] train loss (mean of batch losses): 0.08745457720075335
[7] val loss: 0.07392281799316407
[8] train 

Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.


All 3 operations synced, thanks for waiting!


In [ ]:
folder_names = set(datasets.keys()) | set(results.keys())
base_dir = os.getcwd()
setup_dirs(folder_names, base_dir)
dump_results(datasets, results, base_dir, run)